## **Clone Git Repository**

---

Clone git agar dapat load data langsung dari git repository. Dataset yang digunakan didapat kan dari kaggle: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset?select=Books.csv

In [ ]:
!git clone https://github.com/ziszz/book-recommendation.git

## **Import library yang diperlukan**

---



In [ ]:
import pandas as pd
import numpy as np
import zipfile
import glob
import os
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from zipfile import ZipFile
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## **Data Loading**

---



In [ ]:
books = pd.read_csv('/content/book-recommendation/datasets/Books.csv', encoding='utf-8')
ratings = pd.read_csv('/content/book-recommendation/datasets/Ratings.csv', encoding='utf-8')

In [ ]:
books

In [ ]:
ratings

## **Exploratory Data**

---

### **Variabel-variabel pada dataset adalah sebagai berikut:**

#### **Books.csv**
  * ISBN: Kode pengidentifikasian buku yang bersifat unik.
  * Book-Title: Judul Buku.
  * Book-Author: Nama pengarang buku.
  * Publisher: Pihak penerbit buku.
  * Image-URL-S: URL yang menautkan ke gambar sampul berukuran small.
  * Image-URL-M: URL yang menautkan ke gambar sampul berukuran medium.
  * Image-URL-L: URL yang menautkan ke gambar sampul berukuran large.

#### **Ratings.csv**
  * User-ID: Nomer unik user yang memberikan rating.
  * ISBN: Kode pengidentifikasian buku yang bersifat unik.
  * Book-Rating: Skor dari rating yang diberikan.


### **Menghitung jumlah Buku, User dan Rating**

In [ ]:
print('Jumlah data buku: ', len(books['ISBN'].unique()))
print('Jumlah data user yang memberikan rating: ', len(ratings['User-ID'].unique()))
print('Jumlah data rating pada buku: ', len(ratings['ISBN'].unique()))

### **Mendapatkan info pada data**

Terlihat dari data buku di bawah. Semua kolom data memiliki type data object

In [ ]:
books.info()

Sedangkan, untuk data rating terdapat 2 tipe pada data yaitu numerik (int64) dan object.

In [ ]:
ratings.info()

Pada data buku terdapat 102022 author. Author dengan jumlah buku terbanyak yang terdapat dalam data adalah Agatha Christie dengan jumlah 632 buku.

In [ ]:
author_list = books['Book-Author'].value_counts().keys()
jumlah = books['Book-Author'].value_counts()

author_count = pd.DataFrame({'authors': author_list, 'jumlah': jumlah}).reset_index(drop=True)
author_count

Dari output di bawah, diketahui bahwa nilai maksimum rating adalah 10 dan nilai minimumnya adalah 0. Artinya, skala rating berkisar antara 0 hingga 10.

In [ ]:
rating_list = ratings['Book-Rating'].value_counts().keys()
jumlah = ratings['Book-Rating'].value_counts()

rating_count = pd.DataFrame({'ratings': rating_list, 'jumlah': jumlah}).reset_index(drop=True)
rating_count

### **Mengecek missing value**

Jika dilihat dari data buku dan data rating di bawah. Terdapat sedikit missing value pada data buku, sedangkan pada data rating tidak memiliki missing value.

In [ ]:
books.isnull().sum()

In [ ]:
ratings.isnull().sum()

## **Data Preparation**

---

### **Menghapus data yang tidak diperlukan dan merubah nama column**
Sistem rekomendasi ini hanya memerlukan data author dan rating sebagai fitur untuk model. Beberapa kolom data seperti `'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'` tidak akan digunakan untuk sistem rekomendasi ini. Jadi data tersebut bisa dihapus.

In [ ]:
unused_columns = ['Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']
books.drop(unused_columns, axis=1, inplace=True)

### **Menggabungkan data buku dan rating menjadi satu DataFrame**

In [ ]:
new_books = pd.merge(ratings, books, on='ISBN', how='left')
new_books

### **Menyeleksi data**
Data yang akan digunakan yaitu data buku dengan skor rating di atas 6. Sebab model tidak akan merekomendasikan data dengan rating rendah.

In [ ]:
new_books = new_books[new_books['Book-Rating'] != 0]
new_books = new_books[new_books['Book-Rating'] > 6]
new_books

### **Mengatasi missing value**

In [ ]:
new_books.isnull().sum()

In [ ]:
new_books = new_books.dropna()
new_books.shape

In [ ]:
new_books.isnull().sum()

### **Membuang data duplikat**

In [ ]:
new_books = new_books.drop_duplicates('ISBN')
new_books

## **Model Development**

---

### **Contet-Based Filtering**

In [ ]:
data = new_books
data.sample(5)

#### **Tfid Vectorizer**

In [ ]:
tfid = TfidfVectorizer(token_pattern=r"(?u)\b\w\w+\b\s+\w+")
tfid.fit(data['Book-Author']) 

tfid.get_feature_names() 

#### **Transformasi data kedalam bentuk matriks**

In [ ]:
tfidf_matrix = tfid.fit_transform(data['Book-Author']) 
tfidf_matrix.shape

#### **Menghitung Cosine Similarity**

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix) 
cosine_sim